In [22]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon
from shapely.ops import nearest_points, polygonize
import shapely
from itertools import product, combinations
import math
import warnings
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr
from wpgpDownload.utils.convenience_functions import refresh_csv

In [23]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Tel Aviv']

# idea to convert to dask-pandas and dask-geopandas
# https://towardsdatascience.com/pandas-with-dask-for-an-ultra-fast-notebook-e2621c3769f
# Or with Koalas (Spark-like pandas)

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

In [24]:
# 1. Required preprocess for information extraction

# Let's ignore depreciation warnings
warnings.filterwarnings("ignore")

# Get the city boundaries
bound_df = ox.geocoder.geocode_to_gdf(cities) # gets city boundaries from OSM

# Get unique iso-codes of selected cities (only load country raster once)
unique_iso = iso_countries(bound_df, # Finding the country of the bounded city
                           cities)
print(' ')

print('downloaded:')
# Get raster of countries (if automatic download is preferred (standard))
raster = countries_grids(unique_iso,
                         r'D:\Dumps\WorldPoP_Grids') # custom path, where grid files can be stored without downloading

if prefer dwnl from terminal: 
wpgpDownload download -i ISR --id 5089
 
downloaded:
ISR downloaded 0.01 mns


In [25]:
# 2. Information extraction

# Clip cities from countries, format population grids
population_grids = city_grids_format(bound_df, # city boundaries
                                     unique_iso,
                                     raster, # country raster
                                     cities, 
                                     grid_size = 100)
print(' ')

# Get road networks
road_network = road_networks(cities, # Get 'all' (drive,walk,bike) network
                                 thresholds,
                                 undirected = True)


print(' ')
# Extracting UGS
UGS = urban_greenspace(cities, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

100m resolution grids extraction
Tel Aviv 0.03 mns
 
get road networks from OSM
Tel Aviv done 0.5 mns
 
get urban greenspaces from OSM
Tel Aviv done


In [26]:
# 3. Preprocess information for route finding

# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_network['nodes'], 
                           cities, 
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print(' ')
# Checks all potential suitible combinations (points that fall within max threshold Euclidean distance from the ego)
suitible = suitible_combinations(UGS_entry, 
                                 population_grids, 
                                 road_network['nodes'], # For finding nearest grid entry points
                                 thresholds,
                                 cities,
                                 chunk_size = 10000000) # calculating per chunk of num UGS entry points * num pop_grids
                                                        # Preventing normal PC meltdown, set lower if PC gets stuck
print(' ')
# Checks if grids are already in a UGS
suitible_InOut_UGS = grids_in_UGS (suitible, UGS, population_grids)

get fake UGS entry points
Tel Aviv 0.0 % done 0.0  mns
Tel Aviv 12.7 % done 0.09  mns
Tel Aviv 25.3 % done 0.18  mns
Tel Aviv 38.0 % done 0.26  mns
Tel Aviv 50.6 % done 0.34  mns
Tel Aviv 63.3 % done 0.42  mns
Tel Aviv 75.9 % done 0.5  mns
Tel Aviv 88.6 % done 0.58  mns
Tel Aviv 100 % done 0.69  mns
 
get potential (Euclidean) suitible combinations
Tel Aviv
in chunk 1 / 4 221868 suitible comb.
in chunk 2 / 4 258024 suitible comb.
in chunk 3 / 4 263214 suitible comb.
in chunk 4 / 4 284885 suitible comb.
total combinations within distance 1027991
0.0 % gridentry done 0.0  mns
24.3 % gridentry done 0.37  mns
48.6 % gridentry done 0.75  mns
73.0 % gridentry done 1.13  mns
97.3 % gridentry done 1.5  mns
100 % gridentry done 2.67  mns
 
Check grids within UGS
0 0.0  mns
100 0.07  mns
200 0.13  mns
300 0.19  mns


In [41]:
st = list()
st.append(suitible_InOut_UGS[0].iloc[180000:220000])


In [61]:
# 4. Finding shortest routes.
Routes = route_finding (road_network['graphs'], # graphs of the road networks
               suitible_InOut_UGS, # potential suitible routes with grid-UGS comb. separated in or out UGS.
               road_network['nodes'], 
               road_network['edges'], 
               cities, 
               block_size = 250000, # Chunk to spread dataload.
               nn_iter = 10) # max amount of nearest nodes to be found (both for UGS entry and grid-centroid road entries)


Tel Aviv 1 / 4 range 0 - 250000
0.0 % done 0.01 mns
1.01 % done 0.3 mns
2.02 % done 0.59 mns
3.04 % done 0.8 mns
4.05 % done 0.96 mns
5.06 % done 1.15 mns
6.07 % done 1.3 mns
7.09 % done 1.45 mns
8.1 % done 1.59 mns
9.11 % done 1.73 mns
10.12 % done 1.87 mns
11.14 % done 2.05 mns
12.15 % done 2.28 mns
13.16 % done 2.5 mns
14.17 % done 2.67 mns
15.19 % done 2.86 mns
16.2 % done 3.03 mns
17.21 % done 3.23 mns
18.22 % done 3.42 mns
19.24 % done 3.59 mns
20.25 % done 3.87 mns
21.26 % done 4.38 mns
22.27 % done 4.63 mns
23.29 % done 4.76 mns
24.3 % done 4.9 mns
for 228 routes nearest nodes found
25.31 % pathfinding done 5.03 mns
formatting done 6.58 mns
dissolving done 7.96 mns
dissolving done 7.96 mns
Tel Aviv 2 / 4 range 250000 - 500000
25.31 % done 7.96 mns
26.32 % done 8.1 mns
27.33 % done 8.26 mns
28.35 % done 8.42 mns
29.36 % done 8.57 mns
30.37 % done 8.73 mns
31.38 % done 8.89 mns
32.4 % done 9.03 mns
33.41 % done 9.18 mns
34.42 % done 9.33 mns
35.43 % done 9.49 mns
36.45 % done 9.6

In [93]:
Routes['route summary'][0]

,geometry,way-id,realG_osmid,realP_osmid,way_calc,route_cost,steps,Grid_No,grid_osm,Park_No,Park_entry_No,Parkroad_osmid,Grid_m_centroid,walk_area_m2,Euclidean,geometry_m,real_G-entry,Tcost,gridpark_no
0,"MULTILINESTRING ((34.80809 32.04141, 34.80787 ...",1.0,5666571855,563991156,normal way,1015.637,9.0,2130,5666571855,53,1157,563991156,POINT (3567928.019 4027457.737),298158.324382,731.171505,POINT (3567932.613 4027643.260),185.580,1201.217,2130-53
1,"MULTILINESTRING ((34.80809 32.04141, 34.80787 ...",1.0,5666571855,563991159,normal way,817.385,7.0,1989,5666571855,53,1158,563991159,POINT (3567946.944 4027681.489),288786.071310,593.594564,POINT (3567932.613 4027643.260),40.826,858.211,1989-53
2,"MULTILINESTRING ((34.80809 32.04141, 34.80787 ...",1.0,5666571855,563991159,normal way,817.385,7.0,2061,5666571855,53,1158,563991159,POINT (3567895.698 4027555.848),288786.071310,573.679028,POINT (3567932.613 4027643.260),94.888,912.273,2061-53
3,"MULTILINESTRING ((34.80809 32.04141, 34.80787 ...",1.0,5666571855,563991159,normal way,817.385,7.0,2062,5666571855,53,1158,563991159,POINT (3567979.266 4027583.378),288786.071310,644.745752,POINT (3567932.613 4027643.260),75.910,893.295,2062-53
4,"MULTILINESTRING ((34.80809 32.04141, 34.80787 ...",1.0,5666571855,563991159,normal way,817.385,7.0,2130,5666571855,53,1158,563991159,POINT (3567928.019 4027457.737),288786.071310,643.264282,POINT (3567932.613 4027643.260),185.580,1002.965,2130-53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,POINT (34.77480 32.04010),0.0,1575610986,1575611068,grid in UGS,0.000,0.0,2101,1575611068,208,2564,1575610986,POINT (3564585.433 4026357.320),88595.652375,214.622949,POINT (3564644.477 4026390.264),0.000,0.000,2101-208
9996,POINT (34.77480 32.04010),0.0,1575611010,1575611068,grid in UGS,0.000,0.0,2101,1575611068,208,2565,1575611010,POINT (3564585.433 4026357.320),88236.893190,472.888938,POINT (3564644.477 4026390.264),0.000,0.000,2101-208
9997,POINT (34.77480 32.04010),0.0,1575611030,1575611068,grid in UGS,0.000,0.0,2101,1575611068,208,2566,1575611030,POINT (3564585.433 4026357.320),88595.652375,315.272612,POINT (3564644.477 4026390.264),0.000,0.000,2101-208
9998,POINT (34.77480 32.04010),0.0,1575611073,1575611068,grid in UGS,0.000,0.0,2101,1575611068,208,2567,1575611073,POINT (3564585.433 4026357.320),88182.982440,474.423637,POINT (3564644.477 4026390.264),0.000,0.000,2101-208


In [58]:
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, population_grids, thresholds, cities)

E2SCFA_score['score summary']

300 Tel Aviv
600 Tel Aviv
1000 Tel Aviv


,Tel Aviv
population,"395,909.00"
Sc-access 300,23.02
M-dist 300,115.62
M-area 300,"224,950.32"
M-supply 300,15.59
Sc-norm 300,0.56
Sc-access 600,23.51
M-dist 600,288.68
M-area 600,"194,484.13"
M-supply 600,9.34


In [1]:
def iso_countries(bounds, cities):
    # bound_df = ox.geocoder.geocode_to_gdf(cities)
    # The 'Countries' is a list of iso-countries and descriptions from the package wpgpDownload.utils.isos
    C = pd.DataFrame(Countries)
    start_time = time.time()
    iso_countries = []
    print('if prefer dwnl from terminal: ')
    
    # Check the display name in the city boundaries to get the country name (enabling only specifying city in front)
    for i in bounds['display_name']:
        country = i.rsplit(',')[-1][1:]
        iso = C[C['name'] == country].iloc[0,1]
        # Get unique ISO countries, so all country-grids are only loaded once
        if iso not in iso_countries:
            iso_countries.append(iso)
            
            # List data and extract raster file download string with 2020 population (if download manually is preferred)
            products = Product(iso)
            Results = products.description_contains('people per grid-cell 2020')
            list1 = []
            for p in Results:
                prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
                list1.append(prints)
            print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])
    
    return(iso_countries)

In [2]:
def countries_grids(iso_countries, download_dir = ' '):
    start_time = time.time()
    blocks = []
    for iso in iso_countries:
        # Check if raster files already exist on the system path or a manually specified path
        path1 = os.getcwd() +'\\'+ iso.lower() + '_ppp_2020.tif'
        path2 = download_dir +'\\'+ iso.lower() + '_ppp_2020.tif'
        # First check the manual path
        if os.path.exists(path2): 
            block = gr.from_file(path2)
            blocks.append(block)
        else:
            # Then the system path
            if os.path.exists(path1): 
                block = gr.from_file(path1)
                blocks.append(block)
            else:
                # Otherwise run a suprocess (spr.run) command to download via the terminal in notebook.
                runstr = 'wpgpDownload download -i '+ iso+ ' -f people --datasets'
                p1 = spr.run('wpgpDownload download -i '+ iso+ ' -f people --datasets', 
                                    shell = True, 
                                    capture_output = True)
                # decode the output to a list of available datasets from WorldPoP
                datasets = p1.stdout.decode().rsplit('\n')

                # The first population raster grid (id-sorted) is the general one, without specifying to demographic groups
                for i in enumerate(datasets):
                    if '2020' in i[1]:
                        ds = datasets[i[0]].rsplit('\t')[0]
                        print(ds)
                        # if we found the file, we can stop the loop (we don't need the demograhically specified files)
                        break
                # Construct the download string
                dwnl = 'wpgpDownload download -i '+iso+' --id '+str(ds)
                # Get the specified file (terminal)
                spr.run(dwnl, shell = True)
                # Extract the file
                block = gr.from_file(path1)
                blocks.append(block)
        print(iso,'downloaded', round((time.time() - start_time)/60,2),'mns')
    return(blocks)

In [3]:
# Block 2 population grids extraction
def city_grids_format(bounds, iso_countries, country_grids, cities, grid_size = 100):
    start_time = time.time()
    grids = []
    print(str(grid_size) + 'm resolution grids extraction')
    for i in range(len(cities)):
        C = pd.DataFrame(Countries)
        iso = C[bounds['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
        contains = [j for j, x in enumerate(iso_countries) if x == iso][0]

        # Clip the city from the country
        clipped = country_grids[contains].clip(bounds['geometry'][i])
        clipped = clipped[0].to_geopandas()

        # Get dissolvement_key for dissolvement. 
        clipped['row3'] = np.floor(clipped['row']/(grid_size/100)).astype(int)
        clipped['col3'] = np.floor(clipped['col']/(grid_size/100)).astype(int)
        clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)

        # Dissolve into block by block grids
        popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

        # Get those grids populations and area. Only blocks with population and full blocks
        popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
        popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
        popgrid = popgrid[popgrid['population'] > 0]
        popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]

        # Get centroids and coords
        popgrid['centroid'] = popgrid['geometry'].centroid
        popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
        popgrid['grid_lon'] = popgrid['centroid_m'].x
        popgrid['grid_lat'] = popgrid['centroid_m'].y
        popgrid = popgrid.reset_index()

        minx = popgrid.bounds['minx']
        maxx = popgrid.bounds['maxx']
        miny = popgrid.bounds['miny']
        maxy = popgrid.bounds['maxy']

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
        # I therefore create one polygon
        Poly = []
        for k in range(len(popgrid)):
            Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
        popgrid['geometry'] = Poly

        grids.append(popgrid)

        print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    return(grids)

In [4]:
# Block 3 Road networks
def road_networks (cities, thresholds, undirected = False):
    print('get road networks from OSM')
    start_time = time.time()
    graphs = list()
    road_nodes = list()
    road_edges = list()
    road_conn = list()

    for i in cities:
        # Get graph, road nodes and edges
        graph = ox.graph_from_place(i, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
        #graphs.append(graph)

        road_node, road_edge = ox.graph_to_gdfs(graph)

        # Road nodes format
        road_node = road_node.to_crs(4326)
        road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
        road_node['osmid_var'] = road_node.index
        road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)

        # format road edges
        road_edge = road_edge.to_crs(4326)
        road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
        road_edge = road_edge.reset_index()
        road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
        road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
        
        if undirected == True:
            # Apply one-directional to both for walking
            both = road_edge[road_edge['oneway'] == False]
            one = road_edge[road_edge['oneway'] == True]
            rev = pd.DataFrame()
            rev[['from','to']] = one[['to','from']]
            rev = pd.concat([rev,one.iloc[:,2:]],axis = 1)
            edge_bidir = pd.concat([both, one, rev])
            edge_bidir = edge_bidir.reset_index()
            edge_bidir['oneway'] = False
        else:
            edge_bidir = road_edge

        # Exclude highways and ramps on edges    
        edge_filter = edge_bidir[(edge_bidir['highway'].str.contains('motorway') | 
              (edge_bidir['highway'].str.contains('trunk') & 
               edge_bidir['maxspeed'].astype(str).str.contains(
                   '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
        road_edges.append(edge_filter)

        # Exclude isolated nodes
        fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
        newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
        road_nodes.append(newnodes)

        # Get only necessary road connections columns for network performance
        road_con = edge_filter[['osmid','key','length','geometry']]
        road_con = road_con.set_index('key')

        road_conn.append(road_con)

        # formatting to graph again.
        newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
        edge_filter = edge_filter.set_index(['from','to','keys'])
        edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]

        graph2 = ox.graph_from_gdfs(newnodes, edge_filter)

        graphs.append(graph2)
        print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    return({'graphs':graphs,'nodes':road_nodes,'edges':road_conn,'edges long':road_edges})

In [5]:
# Block 4 city greenspace
def urban_greenspace (cities, thresholds, one_UGS_buf = 25, min_UGS_size = 400):
    print('get urban greenspaces from OSM')
    parks_in_range = list()
    for i in cities:
        gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
        gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
        greenspace = gdf.reset_index()    
        warnings.filterwarnings("ignore")

        green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_UGS_buf).to_crs(4326))
        greenspace['geometry_w_buffer'] = green_buffer
        greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
        greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

        # This function group components in itself that overlap (with the buffer set of 25 metres)
        # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
        W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
        greenspace['components'] = W.component_labels
        parks = greenspace.dissolve('components')

        # Exclude parks below 0.04 ha.
        parks = parks[parks.to_crs(3043).area > min_UGS_size]
        print(i, 'done')
        parks = parks.reset_index()
        parks['geometry_m'] = parks['geometry'].to_crs(3043)
        parks['park_area'] = parks['geometry_m'].area
        parks_in_range.append(parks)
    return(parks_in_range)

In [6]:
# Block 5 park entry points
def UGS_fake_entry(UGS, road_nodes, cities, UGS_entry_buf = 25, walk_radius = 500, entry_point_merge = 0):
    print('get fake UGS entry points')
    start_time = time.time()
    ParkRoads = list()
    for j in range(len(cities)):
        ParkRoad = pd.DataFrame()
        mat = list()
        # For all
        for i in range(len(UGS[j])):
            dist = road_nodes[j]['geometry'].to_crs(3043).distance(UGS[j]['geometry'].to_crs(
                3043)[i])
            buf_nodes = road_nodes[j][(dist < UGS_entry_buf) & (dist > 0)]
            mat.append(list(np.repeat(i, len(buf_nodes))))
            ParkRoad = pd.concat([ParkRoad, buf_nodes])
            if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(UGS[j])*100,1),'% done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        # Park no list conversion
        mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

        # Format
        ParkRoad['Park_No'] = mat_u
        ParkRoad = ParkRoad.reset_index()
        ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
        ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
        
        # Get the road nodes intersecting with the parks' buffer
        ParkRoad = pd.merge(ParkRoad, UGS[j][['geometry','park_area']], left_on = 'Park_No', right_index = True)

        # Get the walkable park size
        ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walk_radius).to_crs(4326).intersection(ParkRoad['geometry_y'])
        ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
        #ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
        ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
                
        # Merge fake UGS entry points if within X meters of each other for better system performance
        # Standard no merging
        ParkRoad = simplify_UGS_entry(ParkRoad, entry_point_merge = 0)
                
        ParkRoads.append(ParkRoad)

        print(cities[j].rsplit(',')[0],'100 % done', 
                                  round((time.time() - start_time) / 60,2),' mns')
    return(ParkRoads)

In [7]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)
def simplify_UGS_entry(fake_UGS_entry, entry_point_merge = 0):
    # Get buffer of nodes close to each other.
    # Get the buffer
    ParkComb = fake_UGS_entry
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(entry_point_merge)

    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels

    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']

    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    return(ParkComb)

In [8]:
# Block 6 grid-parkentry combinations within euclidean threshold distance
def suitible_combinations(UGS_entry, pop_grids, road_nodes, thresholds, cities, chunk_size = 10000000):
    print('get potential (Euclidean) suitible combinations')
    start_time = time.time()
    RoadComb = list()
    for l in range(len(cities)):
        #blockA = block_combinations
        print(cities[l])
        len1 = len(pop_grids[l])
        len2 = len(UGS_entry[l])

        # Reduce the size of combinations per iteration
        len4 = 1
        len5 = len1 * len2
        blockC = len5
        while blockC > chunk_size:
            blockC = len5 / len4
            #print(blockC, len4)
            len4 = len4+1

        # Amount of grids taken per iteration block
        block = round(len1 / len4)

        output = pd.DataFrame()
        # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
        for i in range(len4):
            # Check all grid-park combinations per block
            l1, l2 = range(i*block,(i+1)*block), range(0,len2)
            listed = pd.DataFrame(list(product(l1, l2)))

            # Merge grid and park information
            grid_merged = pd.merge(listed, 
                                   pop_grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                                   left_on = 0, right_index = True)
            node_merged = pd.merge(grid_merged, 
                                   UGS_entry[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                       'share_walked','park_area','walk_area']], 
                                   left_on = 1, right_index = True)

            # Preset index for merging
            node_merged['key'] = range(0,len(node_merged))
            node_merged = node_merged.set_index('key')
            node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

            # Create lists for better computational performance
            glon = list(node_merged['grid_lon'])
            glat = list(node_merged['grid_lat'])
            plon = list(node_merged['park_lon'])
            plat = list(node_merged['park_lat'])

            # Get the euclidean distances
            mat = list()
            for j in range(len(node_merged)):
                mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))

            # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
            mat_df = pd.DataFrame(mat)[(np.array(mat) <= np.max(thresholds))]

            # join the other gravity euclidean scores and other information
            mat_df.columns = ['Euclidean']    
            mat_df = mat_df.join(node_merged)

            output = pd.concat([output, mat_df])

            print('in chunk',(i+1),'/',len4,len(mat_df),'suitible comb.')
        # Renaming columns
        print('total combinations within distance',len(output))

        output.columns = ['Euclidean','Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_lon',
                      'park_lat','parkshare_walked','park_area','walk_area_m2']

        output = output[['Euclidean','Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid','walk_area_m2',
                     'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_area']]

        # Reinstate geographic elements
        output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
        output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
        output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
        output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)

        # Get the nearest entrance point for the grid centroids
        output = gridroad_entry(output, road_nodes[l])

        print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
        RoadComb.append(output)
    return (RoadComb)

In [9]:
def gridroad_entry (suitible_comb, road_nodes):    
    start_time = time.time()
    mat5 = list()
    for i in range(len(suitible_comb)):
        try:
            nearest = int(road_nodes['geometry'].sindex.nearest(suitible_comb['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes['osmid_var'].iloc[nearest])
        except: 
            # sometimes two nodes are the exact same distance, then the first in the list is taken.
            nearest = int(road_nodes['geometry'].sindex.nearest(suitible_comb['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(suitible_comb)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
    # format resulting dataframe
    suitible_comb['grid_osm'] = mat5
    suitible_comb = pd.merge(suitible_comb, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
    suitible_comb['geometry_m'] = gpd.GeoSeries(suitible_comb['geometry'], crs = 4326).to_crs(3043)
    suitible_comb = suitible_comb.reset_index()
    return(suitible_comb)

In [10]:
# Check grids in or out of UGS
def grids_in_UGS (suitible_comb, UGS, pop_grid): 
    start_time = time.time()
    RoadInOut = list()
    for i in range(len(suitible_comb)):
        UGS_geoms = UGS[i]['geometry']
        grid = pop_grid[i]['centroid']
        lst = list()
        print('Check grids within UGS')
        for l in enumerate(UGS_geoms):
            lst.append(grid.intersection(l[1]).is_empty == False)
            if l[0] % 100 == 0: print(l[0], round((time.time() - start_time) / 60,2),' mns')

        dfGrUGS = pd.DataFrame(pd.DataFrame(np.array(lst)).unstack())
        dfGrUGS.columns = ['in_out_UGS']
        merged = pd.merge(suitible_comb[i], dfGrUGS, left_on = ['Grid_No','Park_No'], right_index = True, how = 'left')
        RoadInOut.append(merged)
    return(RoadInOut)    

In [11]:
# WORKS

# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance
def route_finding (graphs, combinations, road_nodes, road_edges, cities, block_size = 250000, nn_iter = 10):

    warnings.filterwarnings("ignore")
    start_time = time.time()

    Routes = list()
    for j in range(len(cities)):
        Graph = graphs[j]
        suit_raw = combinations[j] # iloc to test the iteration speed.
        nodes = road_nodes[j]

        In_UGS = suit_raw[suit_raw['in_out_UGS'] == True] # Check if a grid centroid is in an UGS
        suitible = suit_raw[suit_raw['in_out_UGS'] == False].reset_index(drop = True) # recreate a subsequential index

        block = block_size

        Route_parts = pd.DataFrame()
        len2 = int(np.ceil(len(suitible)/block))
        # Divide in chunks of block for computational load
        for k in range(len2):    
            suitible_chunk = suitible.iloc[k*block:k*block+block]

            parknode = list(suitible_chunk['Parkroad_osmid'])
            gridnode = list(suitible_chunk['grid_osm'])

            s_mat = list([])
            s_mat1 = list([])
            s_mat2 = list([])
            s_mat3 = list([])
            s_mat4 = list([])
            s_mat5 = list([])
            mat_nn = []
            len1 = len(suitible_chunk)

            print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
            for i in range(len(suitible_chunk)):
                try:
                    shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('normal way')
                    s_mat5.append(1)
                except:
                    try:
                        # Check the reverse
                        shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                        s_mat.append(shortest)
                        shortest_to = list(shortest[1:len(shortest)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                        s_mat3.append(list(np.arange(0, len(shortest))))
                        s_mat4.append('reverse way')
                        s_mat5.append(0)
                    except:
                        # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                        # Order in route for nearest node:
                        # 1. gridnode to nearest to the original failed parknode
                        # 2. The reverse of 1.
                        # 3. nearest gridnode to the failed one and route to park
                        # 4. The reverse of 3.

                        len3 = 0
                        alt_route = list([])
                        while len3 < nn_iter and len(alt_route) < 1:

                            len3 = len3 +1
                            #def (suitible_chunk, nodes, nn_i): 
                            # Grid nearest
                            g_geom = nodes[nodes['osmid_var'] == int(suitible_chunk.iloc[i:i+1]['grid_osm'])]['geometry']
                            g_nearest = pd.DataFrame((abs(float(g_geom.x) - nodes['geometry'].x)**2
                            +abs(float(g_geom.y) - nodes['geometry'].y)**2)**(1/2)
                                                    ).join(nodes['osmid_var']).sort_values(0)

                            g_grid = g_nearest.iloc[len3,1]
                            g_park = suitible_chunk.iloc[i]['Parkroad_osmid']

                            p_geom = nodes[nodes['osmid_var'] == int(suitible_chunk.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                            p_nearest = pd.DataFrame((abs(float(p_geom.x) - nodes['geometry'].x)**2
                            +abs(float(p_geom.y) - nodes['geometry'].y)**2)**(1/2)
                                                    ).join(nodes['osmid_var']).sort_values(0)

                            p_grid = suitible_chunk.iloc[i]['grid_osm']
                            p_park = p_nearest.iloc[len3,1]

                            try:
                                alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                                  'travel_dist', method = 'dijkstra'))
                                s_mat4.append(str(len3)+'grid > n-park')
                                s_mat5.append(1)
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                      'travel_dist', method = 'dijkstra'))
                                    s_mat4.append(str(len3)+'n-park > grid')
                                    s_mat5.append(0)
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                        s_mat4.append(str(len3)+'n-grid > park')
                                        s_mat5.append(1)
                                    except:
                                        try:
                                            alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                              'travel_dist', method = 'dijkstra'))
                                            s_mat4.append(str(len3)+'park > n-grid')
                                            s_mat5.append(0)
                                        except:
                                            if len3 == nn_iter:
                                                #print(i+block*k,i+block*k,
                                                #      'No route between grid and park-entry and their both 10 alternatives')
                                                pass
                                            pass
                        #print(len(alt_route))
                        if len(alt_route) == 0: 
                            alt = alt_route 
                        else: 
                            alt = alt_route[0]
                        len4 = len(alt)
                        #print(len4)
                        #mat_nn = []
                        if len4 > 0:
                            #print('for index',i+block*k,'nearest node found between', 
                            #                       alt[0],'and',alt[-1])
                            mat_nn.append(i+block*k)
                            s_mat.append(alt)
                            shortest_to = list(alt[1:len(alt)])
                            shortest_to.append(-1)
                            s_mat1.append(shortest_to)
                            s_mat2.append(list(np.repeat(i+block*k,len4)))
                            s_mat3.append(list(np.arange(0, len4)))
                        else:
                            s_mat.append(-1)
                            s_mat1.append(-1)
                            s_mat2.append(i+block*k)
                            s_mat3.append(-1)
                            s_mat4.append('no way')
                            s_mat5.append(2)
                            print(i+block*k,'No route',nn_iter)

                if i % 10000 == 0: print(round((i+block*k)/len(suitible)*100,2),'% done',
                                         round((time.time() - start_time) / 60,2),'mns')
            print('for', len(mat_nn),'routes nearest nodes found')

            print(round((i+block*k)/len(suitible)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')

            # Formats route information by route and step (detailed)
            routes = route_formatting(s_mat, s_mat1, s_mat2, s_mat3, road_edges[j])
            print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
            
            # Summarizes information by route
            routes2 = route_summarization(routes, suitible_chunk, road_nodes[j], s_mat4, s_mat5)
            print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
            
            print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
            Route_parts = pd.concat([Route_parts, routes2])

        # Format grids in UGS to enable smooth df concat
        In_UGS = In_UGS.set_geometry(In_UGS['Grid_coords_centroid'])
        In_UGS = In_UGS[['geometry','Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                   'Grid_m_centroid','walk_area_m2',
                                   'Euclidean','geometry_m']]

        In_UGS['realG_osmid'] = suit_raw['Parkroad_osmid']
        In_UGS['realP_osmid'] = suit_raw['grid_osm']
        In_UGS['way_calc'] = 'grid in UGS'

        Route_parts = pd.concat([Route_parts,In_UGS])
        Route_parts = Route_parts.reset_index(drop = True)

        Route_parts['gridpark_no'] = Route_parts['Grid_No'].astype(str) +'-'+ Route_parts['Park_No'].astype(str)

        # All fill value 0 because no routes are calculated for grid centroids in UGSs
        to_fill = ['way-id','route_cost','steps','real_G-entry','Tcost']                                   
        Route_parts[to_fill] = Route_parts[to_fill].fillna(0)  
            
        Routes.append(Route_parts)
    return(Routes)

In [43]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance
def route_finding (graphs, combinations, road_nodes, road_edges, cities, block_size = 250000, nn_iter = 10):

    warnings.filterwarnings("ignore")
    start_time = time.time()

    Routes = list()
    for j in range(len(cities)):
        Graph = graphs[j]
        suit_raw = combinations[j] # iloc to test the iteration speed.
        nodes = road_nodes[j]

        In_UGS = suit_raw[suit_raw['in_out_UGS'] == True] # Check if a grid centroid is in an UGS
        suitible = suit_raw[suit_raw['in_out_UGS'] == False].reset_index(drop = True) # recreate a subsequential index

        block = block_size

        Route_parts = pd.DataFrame()
        len2 = int(np.ceil(len(suitible)/block))
        # Divide in chunks of block for computational load
        for k in range(len2):    
            suitible_chunk = suitible.iloc[k*block:k*block+block]

            parknode = list(suitible_chunk['Parkroad_osmid'])
            gridnode = list(suitible_chunk['grid_osm'])

            s_mat = list([])
            s_mat1 = list([])
            s_mat2 = list([])
            s_mat3 = list([])
            s_mat4 = list([])
            s_mat5 = list([])
            mat_nn = []
            len1 = len(suitible_chunk)

            print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
            for i in range(len(suitible_chunk)):
                try:
                    shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('normal way')
                    s_mat5.append(1)
                except:
                    try:
                        # Check the reverse
                        shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                        s_mat.append(shortest)
                        shortest_to = list(shortest[1:len(shortest)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                        s_mat3.append(list(np.arange(0, len(shortest))))
                        s_mat4.append('reverse way')
                        s_mat5.append(0)
                    except:
                        # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                        nn_route_finding(Graph, suitible_chunk, nodes, s_mat, s_mat1, s_mat2, s_mat3
                                             s_mat4, s_mat5, i, block, k, nn_iter)
                        
                if i % 10000 == 0: print(round((i+block*k)/len(suitible)*100,2),'% done',
                                         round((time.time() - start_time) / 60,2),'mns')
            print('for', len(mat_nn),'routes nearest nodes found')

            print(round((i+block*k)/len(suitible)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')

            # Formats route information by route and step (detailed)
            routes = route_formatting(s_mat, s_mat1, s_mat2, s_mat3, road_edges[j])
            print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
            
            # Summarizes information by route
            routes2 = route_summarization(routes, suitible_chunk, road_nodes[j], s_mat4, s_mat5)
            print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
            
            print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
            Route_parts = pd.concat([Route_parts, routes2])

        # Format grids in UGS to enable smooth df concat
        In_UGS = In_UGS.set_geometry(In_UGS['Grid_coords_centroid'])
        In_UGS = In_UGS[['geometry','Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                   'Grid_m_centroid','walk_area_m2',
                                   'Euclidean','geometry_m']]

        In_UGS['realG_osmid'] = suit_raw['Parkroad_osmid']
        In_UGS['realP_osmid'] = suit_raw['grid_osm']
        In_UGS['way_calc'] = 'grid in UGS'

        Route_parts = pd.concat([Route_parts,In_UGS])
        Route_parts = Route_parts.reset_index(drop = True)

        Route_parts['gridpark_no'] = Route_parts['Grid_No'].astype(str) +'-'+ Route_parts['Park_No'].astype(str)

        # All fill value 0 because no routes are calculated for grid centroids in UGSs
        to_fill = ['way-id','route_cost','steps','real_G-entry','Tcost']                                   
        Route_parts[to_fill] = Route_parts[to_fill].fillna(0)  
            
        Routes.append(Route_parts)
    return(Routes)

In [59]:
def nn_route_finding(graph, suitible_chunk, nodes, mat_from, mat_to, mat_route, mat_step,
                                             mat_way, mat_wbin, i, block, k, nn_iter):
                        
    # Order in route for nearest node:
    # 1. gridnode to nearest to the original failed parknode
    # 2. The reverse of 1.
    # 3. nearest gridnode to the failed one and route to park
    # 4. The reverse of 3.
                        
    gridosm = suitible_chunk['grid_osm']
    UGSosm = suitible_chunk['Parkroad_osmid']
    nodeosm = nodes['osmid_var']
    nodegeom = nodes['geometry']
                        
    len3 = 0
    alt_route = list([])
    while len3 < nn_iter and len(alt_route) < 1:

        len3 = len3 +1
                            
        nn = nn_finding(gridosm, UGSosm, nodeosm, nodegeom, nodes, i, len3)

        nn_routing (graph, nn['currUGS'], nn['nearUGS'], nn['currgrid'], nn['neargrid'], 
                                        mat_way, mat_wbin, len3, alt_route)
    if len(alt_route) == 0: 
        alt = alt_route 
    else: 
        alt = alt_route[0]
    len4 = len(alt)
    if len4 > 0:
        mat_nn.append(i+block*k)
        mat_from.append(alt)
        shortest_to = list(alt[1:len(alt)])
        shortest_to.append(-1)
        mat_to.append(shortest_to)
        mat_route.append(list(np.repeat(i+block*k,len4)))
        mat_step.append(list(np.arange(0, len4)))
    else:
        mat_from.append(-1)
        mat_to.append(-1)
        mat_route.append(i+block*k)
        mat_step.append(-1)
        mat_way.append('no way')
        mat_wbin.append(2)
        print(i+block*k,'No route',nn_iter)

In [60]:
def nn_finding (gridosm, UGSosm, nodeosm, nodegeom, nodes, i, nn_i): 
    # Grid nearest
    g_geom = nodegeom[nodeosm == int(gridosm[i:i+1])]
    g_nearest = pd.DataFrame((abs(float(g_geom.x) - nodegeom.x)**2
    +abs(float(g_geom.y) - nodegeom.y)**2)**(1/2)
                            ).join(nodeosm).sort_values(0)

    g_grid = g_nearest.iloc[nn_i,1]
    g_park = list(UGSosm)[i]
        
    p_geom = nodegeom[nodeosm == int(UGSosm[i:i+1])]
    p_nearest = pd.DataFrame((abs(float(p_geom.x) - nodegeom.x)**2
    +abs(float(p_geom.y) - nodegeom.y)**2)**(1/2)
                            ).join(nodeosm).sort_values(0)

    p_grid = list(gridosm)[i]
    p_park = p_nearest.iloc[nn_i,1]
    return({'currUGS':p_grid, 'nearUGS':p_park,'currgrid':g_park, 'neargrid':g_grid})

In [56]:
def nn_routing (graph, curr_UGS, near_UGS, curr_grid, near_grid, mat_way, mat_wbin, nn_i, found_route):
    try:
        found_route.append(nx.shortest_path(graph, curr_UGS, near_UGS, 
                                          'travel_dist', method = 'dijkstra'))
        mat_way.append(str(nn_i)+'grid > n-park')
        mat_wbin.append(1)
    except:
        try:
            found_route.append(nx.shortest_path(graph, near_UGS, curr_UGS, 
                                              'travel_dist', method = 'dijkstra'))
            mat_way.append(str(nn_i)+'n-park > grid')
            mat_wbin.append(0)
        except:
            try:
                found_route.append(nx.shortest_path(graph, curr_grid, near_grid, 
                                                  'travel_dist', method = 'dijkstra'))
                mat_way.append(str(nn_i)+'n-grid > park')
                mat_wbin.append(1)
            except:
                try:
                    found_route.append(nx.shortest_path(graph, near_grid, curr_grid, 
                                                      'travel_dist', method = 'dijkstra'))
                    mat_way.append(str(nn_i)+'park > n-grid')
                    mat_wbin.append(0)
                except:
                    pass

In [16]:
def route_formatting(mat_from, mat_to, mat_route, mat_step, road_edges):
    # Unpack lists
    s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_from) for i in b]
    s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_to) for i in b]
    s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_route) for i in b]
    s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_step) for i in b]

    # Format df
    routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
    routes.columns = ['from','to','route','step']
    mat_key = list([])
    for n in range(len(routes)):
        mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
    routes['key'] = mat_key
    routes = routes.set_index('key')

    # Add route information
    routes = routes.join(road_edges, how = 'left')
    routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
    routes = routes.sort_values(by = ['route','step'])
    return(routes)

In [17]:
def route_summarization(routes, suitible_comb, road_nodes, mat_way, mat_wbin):
    # dissolve route
    routes2 = routes[['route','geometry']].dissolve('route')

    # get used grid- and parkosm. Differs at NN-route.
    route_reset = routes.reset_index()
    origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
    origin = origin.reset_index().iloc[:,-1]
    dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
    dest = dest.reset_index().iloc[:,-1]

    # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
    routes2['way-id'] = mat_wbin
    routes2['realG_osmid'] = np.where(routes2['way-id'] == 1, origin, dest)
    routes2['realP_osmid'] = np.where(routes2['way-id'] == 1, dest, origin)
    routes2['way_calc'] = mat_way

    # get route cost, steps, additional information.
    routes2['route_cost'] = routes.groupby('route')['length'].sum()
    routes2['steps'] = routes.groupby('route')['step'].max()
    routes2['index'] = suitible_comb.index
    routes2 = routes2.set_index(['index'])
    routes2.index = routes2.index.astype(int)
    routes2 = pd.merge(routes2, suitible_comb[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                          'Grid_m_centroid','walk_area_m2','Euclidean']],
                                            left_index = True, right_index = True)
    routes2 = pd.merge(routes2, road_nodes['geometry_m'], how = 'left', left_on = 'realG_osmid', right_index = True)
    # calculate distance of used road-entry for grid-centroid.
    routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043).distance(routes2['geometry_m']),3)
                                    
    # Calculcate total route cost for the four gravity variants
    routes2['Tcost'] = routes2['route_cost'] + routes2['real_G-entry']
    return(routes2)

In [18]:
def min_gridUGS_comb (routes, grids, UGS):
    gp_nearest = []
    for i in range(len(routes)):
        gp_nn = routes[i][routes[i]['Tcost'] <= max(thresholds)]
        gp_nn = pd.merge(gp_nn, grids[i]['population'], left_on='Grid_No', right_index = True)
        gp_nn = pd.merge(gp_nn, UGS[i]['park_area'], left_on = 'Park_No', right_index = True)
        gp_nn = gp_nn.reset_index()

        gp_nn = gp_nn.iloc[gp_nn.groupby('gridpark_no')['Tcost'].idxmin()]
        gp_nn.index.name = 'idx'
        gp_nn = gp_nn.sort_values('idx')
        gp_nn = gp_nn.reset_index()
        gp_nearest.append(gp_nn)
    gp_nearest[0].sort_values('Grid_No')
    return(gp_nearest)

In [19]:
def E2SCFA_scores(min_gridUGS_comb, grids, thresholds, cities):
    pd.options.display.float_format = '{:20,.2f}'.format
    E2SFCA_cities = []
    E2SFCA_summary = pd.DataFrame()
    for i in range(len(cities)):
        E2SFCA_score = grids[i][['population','geometry']]
        for j in range(len(thresholds)):
            subset = min_gridUGS_comb[i][min_gridUGS_comb[i]['Tcost'] <= thresholds[j]]

            # use gussian distribution: let v= 923325, then the weight for 800m is 0.5
            v = -thresholds[j]**2/np.log(0.5)

            # add a column of weight: apply the decay function on distance
            subset['weight'] = np.exp(-(subset['Tcost']**2/v)).astype(float)
            subset['pop_weight'] = subset['weight'] * subset['population']

            # get the sum of weighted population each green space has to serve.
            s_w_p = pd.DataFrame(subset.groupby('Park_No').sum('pop_weight')['pop_weight'])

            # delete other columns, because they are useless after groupby
            s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
            middle = pd.merge(subset,s_w_p, how = 'left', on = 'Park_No' )

            # calculate the supply-demand ratio for each green space
            middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']

            # caculate the accessbility score for each green space that each population grid cell could reach
            middle['Sc-access'] = middle['weight'] * middle['green_supply']
            # add the scores for each population grid cell
            pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])

            # calculate the mean distance of all the green space each population grid cell could reach
            mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
            pop_score_df['M-dist'] = mean_dist

            # calculate the mean area of all the green space each population grid cell could reach
            mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
            pop_score_df['M-area'] = mean_area

            # calculate the mean supply_demand ratio of all the green space each population grid cell could reach
            mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
            pop_score_df['M-supply'] = mean_supply

            pop_score = pop_score_df

            pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
            pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']

            pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
            pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
            E2SFCA_score = E2SFCA_score.join(pop_score_df, how = 'left')

            print(thresholds[j], cities[i])

        E2SFCA_score = E2SFCA_score.fillna(0)
        pop_sum = pd.Series(E2SFCA_score['population'].sum()).astype(int)
        pop_sum.index = ['population']
        mean_metrics = E2SFCA_score.loc[:, E2SFCA_score.columns != 'population'].mean()
        E2SFCA_sum = pd.concat([pop_sum, mean_metrics])
        E2SFCA_summary = pd.concat([E2SFCA_summary, E2SFCA_sum], axis = 1)
        E2SFCA_cities.append(E2SFCA_score)
        E2SFCA_score.loc[:, E2SFCA_score.columns != 'geometry'].to_csv('D:/Dumps/M2SFCA-OD/Scores/'+cities[i]+'.csv')
    E2SFCA_summary.columns = cities
    E2SFCA_summary.to_csv('D:/Dumps/M2SFCA-OD/Scores/all_cities.csv')
    E2SFCA_summary
    return({'score summary':E2SFCA_summary,'score detail':E2SFCA_cities})

In [20]:
def ranked_route_scoring (routes, UGS_entry, cities, var_abbr, ranks):
    start_time = time.time()
    ranked_routes = list()
    for k in range(len(cities)):
        str1 = var_abbr+'_Tcost'
        # Get the size to meters
        UGS_entry[k]['park_size_walkable_m'] = UGS_entry[k]['park_size_walkable'].to_crs(3043)

        # Drop 'no way' routes
        Rclean = routes[k][routes[k]['way_calc'] != 'no way'].reset_index(drop = True)

        # Create a rank per grid-UGS combination which (fake) entry is the closest
        Rclean['rank'] = Rclean.groupby('gridpark_no')[var_abbr+'_Tcost'].rank(method = 'first').astype(int)

        # Get the UGS walkable radius (standard on 500m from the fake entrance)
        mercl = pd.merge(Rclean, UGS_entry[k]['park_size_walkable_m'], left_on = 'Park_entry_No', right_index = True)

        # Sort for slightly better performance
        mercl = mercl.sort_values(['rank','gridpark_no'])
                                                                    # Radii are walkable area, max distance by entry point.
        # Get a df with unique grid-park combinations as index       # ranks = by grid-UGS comb. rank entry points by route cost
        df = pd.DataFrame(index = Rclean['gridpark_no'].sort_values().unique()) # dissolved UGS radii of current and previous ranks
        df2 = pd.DataFrame(index = Rclean['gridpark_no'].sort_values().unique()) # inflation factor over raw route total cost
        df3 = pd.DataFrame(index = Rclean['gridpark_no'].sort_values().unique()) # radii of ranks
        df4 = pd.DataFrame(index = Rclean['gridpark_no'].sort_values().unique()) # clipped UGS radii from previous (clipped) ranks
        maxrank = max(Rclean['rank'])

        # Enter info for the first rank.
        df[1] = mercl[['gridpark_no','park_size_walkable_m']].set_geometry('park_size_walkable_m').dissolve(by = 'gridpark_no')
        df3[1] = mercl[mercl['rank'] == 1][['gridpark_no','park_size_walkable_m']].set_index('gridpark_no')
        df4[1] = df[1]
        df2[1] = None

        if ranks <= maxrank: iterations = ranks
        else: iterations = maxrank
        print('rank',1,round((time.time() - start_time) / 60,2), 'mns ('+str(len(df))+' routes)')

        for i in range(1,iterations):
            ranked = mercl[mercl['rank'] == i+1][['gridpark_no','park_size_walkable_m']] # Get current rank
            ranked = ranked.sort_values('gridpark_no').set_geometry('park_size_walkable_m')
            ranked = ranked.set_index('gridpark_no')
            df3[i+1] = ranked['park_size_walkable_m']

            series = gpd.GeoSeries(df[i]).difference(gpd.GeoSeries(df3[i+1])) # Gets difference of dissolved radii and current 
            df[i+1] = only_polygons(series) # removes linestrings                 walkable UGS area
            series1 = gpd.GeoSeries(only_polygons(df[i+1]))
            clipped = only_polygons(series1.clip(ranked)) # Get the area not yet served.
            df2[i+1] = clipped.area / UGS_entry[k]['walk_area'].median() # Get the inflation factor by comparing it against
            df4[i+1] = clipped                                           # the median UGS size used earlier.
            print('rank',i+1, round((time.time() - start_time) / 60,2), 'mns ('+str(len(ranked))+' routes)')

        # Apply inflation factors to route cost.
        df2_unstacked = pd.DataFrame(df2.fillna(0).unstack())
        mercl = pd.merge(mercl,df2_unstacked, left_on = ['rank','gridpark_no'], right_index = True)
        mercl['grav2_Tcost'] = np.where(mercl['rank'] == 1,mercl['grav2_Tcost'],mercl['raw_Tcost'] / (mercl[0] ** (1/2)))
        mercl['grav3_Tcost'] = np.where(mercl['rank'] == 1,mercl['grav3_Tcost'],mercl['raw_Tcost'] / (mercl[0] ** (1/3)))
        mercl['grav5_Tcost'] = np.where(mercl['rank'] == 1,mercl['grav5_Tcost'],mercl['raw_Tcost'] / (mercl[0] ** (1/5)))

        mercl = mercl[mercl.columns[mercl.columns != 0]]
        ranked_routes.append(mercl)
    return(ranked_routes)

In [21]:
def only_polygons (geoseries):
    geom_df = series[series.geom_type == 'GeometryCollection']
    non_geom_df = series[series.geom_type != 'GeometryCollection']
    l2 = list()
    for j in range(len(geom_df)):
        l1 = list()
        for i in geom_df[j]: 
            if ((i.geom_type == 'Polygon') |
            (i.geom_type == 'MultiPolygon')):
                l1.append(i)
        l2.append(MultiPolygon(l1))
    done_df = gpd.GeoSeries(l2)
    done_df.index = geom_df.index
    done_df = gpd.GeoSeries(pd.concat([non_geom_df, done_df]))
    return(done_df)

In [24]:
print(round((time.time() - start) / 60,2),'mns')

0.2 mns
